In [1]:
import pandas as pd #导入Pandas
import numpy as np #导入Numpy
import jieba #导入结巴分词

from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU


/opt/gongxf/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
neg=pd.read_excel('data/neg.xls',header=None,index=None)
pos=pd.read_excel('data/pos.xls',header=None,index=None) #读取训练语料完毕
pos['mark']=1
neg['mark']=0 #给训练语料贴上标签
pn=pd.concat([pos,neg],ignore_index=True) #合并语料
neglen=len(neg)
poslen=len(pos) #计算语料数目

In [4]:
neg.head(3)

,0,mark
0,做为一本声名在外的流行书，说的还是广州的外企，按道理应该和我的生存环境差不多啊。但是一看之下...,0
1,作者有明显的自恋倾向，只有有老公养不上班的太太们才能像她那样生活。很多方法都不实用，还有抄袭...,0
2,作者完全是以一个过来的自认为是成功者的角度去写这个问题，感觉很不客观。虽然不是很喜欢，但是，...,0


In [5]:
pos.head(3)

,0,mark
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...,1
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1


In [6]:
neglen,poslen

(10428, 10677)

In [7]:
cw = lambda x: list(jieba.cut(x)) #定义分词函数
pn['words'] = pn[0].apply(cw)

comment = pd.read_excel('data/sum.xls') #读入评论内容
#comment = pd.read_csv('a.csv', encoding='utf-8')
comment = comment[comment['rateContent'].notnull()] #仅读取非空评论
comment['words'] = comment['rateContent'].apply(cw) #评论分词

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.996 seconds.
Prefix dict has been built succesfully.


In [20]:
len(comment['words'])

11177

In [18]:
# pn['words']

In [14]:
d2v_train = pd.concat([pn['words'], comment['words']], ignore_index = True)

In [22]:
len(pn['words']),len(d2v_train)

(21105, 32282)

In [23]:
w = [] #将所有词语整合在一起
for i in d2v_train:
    w.extend(i)

In [24]:
w

['做',
 '父母',
 '一定',
 '要',
 '有',
 '刘墉',
 '这样',
 '的',
 '心态',
 '，',
 '不断',
 '地',
 '学习',
 '，',
 '不断',
 '地',
 '进步',
 '，',
 '不断',
 '地',
 '给',
 '自己',
 '补充',
 '新鲜血液',
 '，',
 '让',
 '自己',
 '保持',
 '一颗',
 '年轻',
 '的',
 '心',
 '。',
 '我',
 '想',
 '，',
 '这',
 '是',
 '他',
 '能',
 '很',
 '好',
 '的',
 '和',
 '孩子',
 '沟通',
 '的',
 '一个',
 '重要',
 '因素',
 '。',
 '读',
 '刘墉',
 '的',
 '文章',
 '，',
 '总能',
 '让',
 '我',
 '看到',
 '一个',
 '快乐',
 '的',
 '平易近人',
 '的',
 '父亲',
 '，',
 '他',
 '始终',
 '站',
 '在',
 '和',
 '孩子',
 '同样',
 '的',
 '高度',
 '，',
 '给',
 '孩子',
 '创造',
 '着',
 '一个',
 '充满',
 '爱',
 '和',
 '自由',
 '的',
 '生活',
 '环境',
 '。',
 '很',
 '喜欢',
 '刘墉',
 '在',
 '字里行间',
 '流露出',
 '的',
 '做',
 '父母',
 '的',
 '那种',
 '小',
 '狡黠',
 '，',
 '让',
 '人',
 '总是',
 '忍俊不禁',
 '，',
 '父母',
 '和',
 '子女',
 '之间',
 '有时候',
 '也',
 '是',
 '一种',
 '战斗',
 '，',
 '武力',
 '争斗',
 '过于',
 '低级',
 '了',
 '，',
 '智力',
 '较量',
 '才',
 '更',
 '有',
 '趣味',
 '。',
 '所以',
 '，',
 '做',
 '父母',
 '的',
 '得',
 '加把劲',
 '了',
 '，',
 '老',
 '思想',
 '老',
 '观念',
 '注定',
 '会',
 '一败涂地',
 '，',
 '生命不息',
 '，',
 '学习',
 

In [25]:
dict = pd.DataFrame(pd.Series(w).value_counts()) #统计词的出现次数

In [27]:
# dict

In [28]:
dict['id']=list(range(1,len(dict)+1))

In [30]:
# dict

In [31]:
get_sent = lambda x: list(dict['id'][x])

In [32]:
pn['sent'] = pn['words'].apply(get_sent) 

In [35]:
pn

,0,mark,words,sent
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...,1,"[做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,...","[156, 522, 266, 44, 18, 14623, 98, 2, 1777, 1,..."
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,1,"[作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,...","[144, 4026, 19644, 2876, 2, 838, 1, 739, 795, ..."
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,1,"[作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ...","[144, 7742, 5425, 1046, 4644, 18822, 368, 23, ..."
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,1,"[作者, 在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, ...","[144, 13, 18554, 10378, 404, 31, 4, 1646, 9074..."
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,1,"[作者, 在, 少年, 时即, 喜, 阅读, ，, 能, 看出, 他, 精读, 了, 无数,...","[144, 13, 3176, 28236, 12053, 287, 1, 67, 2265..."
5,作者有一种专业的谨慎，若能有幸学习原版也许会更好，简体版的书中的印刷错误比较多，影响学者理解...,1,"[作者, 有, 一种, 专业, 的, 谨慎, ，, 若能, 有幸, 学习, 原版, 也许, ...","[144, 18, 355, 599, 2, 6478, 1, 9722, 8037, 41..."
6,作者用诗一样的语言把如水般清澈透明的思想娓娓道来，像一个经验丰富的智慧老人为我们解开一个又一...,1,"[作者, 用, 诗, 一样, 的, 语言, 把, 如水般, 清澈, 透明, 的, 思想, 娓...","[144, 31, 4350, 130, 2, 517, 104, 28381, 5106,..."
7,作者提出了一种工作和生活的方式，作为咨询界的元老，不仅能提出理念，而且能够身体力行地实践，并...,1,"[作者, 提出, 了, 一种, 工作, 和, 生活, 的, 方式, ，, 作为, 咨询, 界...","[144, 739, 4, 355, 368, 23, 206, 2, 607, 1, 35..."
8,作者妙语连珠，将整个60-70年代用层出不穷的摇滚巨星与自身故事紧紧相连什么是乡愁？什么是摇...,1,"[作者, 妙语连珠, ，, 将, 整个, 60, -, 70, 年代, 用, 层出不穷, 的...","[144, 22148, 1, 278, 816, 1565, 158, 2210, 121..."
9,作者逻辑严密，一气呵成。没有一句废话，深入浅出，循循善诱，环环相扣。让平日里看到指标图释就头...,1,"[作者, 逻辑, 严密, ，, 一气呵成, 。, 没有, 一句, 废话, ，, 深入浅出, ...","[144, 2257, 4130, 1, 7449, 3, 21, 801, 2405, 1..."


In [48]:
x = np.array(list(pn['sent']))[::2] #训练集
y = np.array(list(pn['mark']))[::2]
xt = np.array(list(pn['sent']))[1::2] #测试集
yt = np.array(list(pn['mark']))[1::2]
xa = np.array(list(pn['sent'])) #全集
ya = np.array(list(pn['mark']))

In [49]:
len(x)

10553

In [50]:
maxlen = 50

In [51]:
model = Sequential()
model.add(Embedding(input_dim=len(dict)+1, output_dim=256, input_length=maxlen))
model.add(LSTM(128)) # try using a GRU instead, for fun
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [52]:
model.compile(loss='binary_crossentropy', optimizer='adam', class_mode="binary")

In [55]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [53]:
model.fit(x, y, batch_size=16, nb_epoch=1) #训练时间为若干个小时

/opt/gongxf/anaconda3/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected embedding_3_input to have shape (50,) but got array with shape (1,)